<a href="https://colab.research.google.com/github/solobala/RMSL_9/blob/main/Lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дополнительные ссылки:
1. https://sales-generator.ru/blog/rekomendatelnye-sistemy/
2. https://habr.com/ru/companies/prequel/articles/567648/
3. https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101
4. https://habr.com/ru/companies/prequel/articles/573880/

https://bit.ly/RSML-3-colab

## Коллаборативная фильтрация

### item-to-item

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-07-13 09:55:59--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 64.233.183.101, 64.233.183.138, 64.233.183.113, ...
Connecting to drive.google.com (drive.google.com)|64.233.183.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uomlndc09hu6jqie14i6vjv0ug6bn7co/1689242100000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=2ff9e8e3-f65e-4854-aaf8-48b5cbb39591 [following]
--2023-07-13 09:56:00--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uomlndc09hu6jqie14i6vjv0ug6bn7co/1689242100000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=2ff9e8e3-f65e-4854-aaf8-48b5cbb39591
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 142.251.6.132, 2607:f8b0:4001:c5a::84
Connecting to doc-0g-

In [3]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [4]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
movies.shape, ratings.shape

((9742, 3), (100836, 4))

In [10]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [11]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [12]:
movies_with_ratings.shape

(100836, 6)

In [14]:
movies_with_ratings.title.value_counts() # кол-во рейтингов на каждый фильм

Forrest Gump (1994)                      329
Shawshank Redemption, The (1994)         317
Pulp Fiction (1994)                      307
Silence of the Lambs, The (1991)         279
Matrix, The (1999)                       278
                                        ... 
We're Back! A Dinosaur's Story (1993)      1
American Hardcore (2006)                   1
Shanghai Surprise (1986)                   1
Let's Get Harry (1986)                     1
Andrew Dice Clay: Dice Rules (1991)        1
Name: title, Length: 9719, dtype: int64

In [16]:
movies_with_ratings.userId.value_counts() # кол-во рейтингов, проставленных каждым пользователем

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
53       20
207      20
431      20
442      20
189      20
Name: userId, Length: 610, dtype: int64

In [17]:
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

610

In [18]:
movie_vector = {} # Строим вектор для фильма длиной 610 по к-ву пользователей

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users) # заполняем нулями

    for i in range(len(group.userId.values)):  # по каждому тз пользователей ставившиз рейтинг фильму
        u = group.userId.values[i] # Это идентификатор пользователя
        r = group.rating.values[i] # это рейтинг, который он поставил
        movie_vector[movie][int(u - 1)] = r # u-1, т.к это нумерация столбцов вектора, которая начинаетися с нуля, а пользователи нумеруются с 1

100%|██████████| 9719/9719 [00:03<00:00, 3006.62it/s]


In [19]:
movie_vector['Toy Story (1995)']

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ,
       3. , 0. , 0. , 0. , 5. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. ,
       5. , 0. , 0. , 5. , 3. , 4. , 5. , 0. , 0. , 0. , 3. , 0. , 0. ,
       0. , 3. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 0. ,
       4. , 0. , 2.5, 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0.5, 0. , 4. ,
       0. , 0. , 0. , 2.5, 0. , 0. , 0. , 4. , 0. , 0. , 3. , 3. , 4. ,
       0. , 3. , 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0. , 0. , 4. , 0. ,
       0. , 0. , 4. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 3. ,
       0. , 2. , 0. , 3. , 4. , 0. , 4. , 0. , 0. , 3. , 4. , 0. , 0. ,
       3.5, 5. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 2. , 0. , 3. , 4. ,
       0. , 0. , 4.5, 4. , 4. , 0. , 0. , 0. , 0. , 5. , 3.5, 0. , 4.5,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 4. , 0. , 0.

In [20]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [21]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    if key == my_fav_film:
        continue

    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

len(distances)

100%|██████████| 9719/9719 [00:00<00:00, 24281.37it/s]


9718

In [28]:
sorted(distances)[-10:] # Это последние 10 элементов отсортированного списка дистанций

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [29]:
best_indexes = np.argsort(distances)[:10] # А это идентификаторы соотвествующих фильмов
best_indexes

array([5511, 5571,  420, 4736, 5203,  405, 6864, 5204, 4322, 4737])

In [30]:
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

('Matrix, The (1999)', 0.28606257328028306)
('Memento (2000)', 0.33040704494686757)
('American History X (1998)', 0.3509458374819995)
('Kill Bill: Vol. 1 (2003)', 0.360262413456623)
('Lord of the Rings: The Fellowship of the Ring, The (2001)', 0.36425580638414523)
('American Beauty (1999)', 0.3744505150386159)
('Pulp Fiction (1994)', 0.37678014788325576)
('Lord of the Rings: The Return of the King, The (2003)', 0.37798413607448667)
('Inception (2010)', 0.3845825728533704)
('Kill Bill: Vol. 2 (2004)', 0.38565615909903594)


### Surprise

https://surpriselib.com/

- Алгоритмы https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [31]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3097739 sha256=75a694492d271b7a0da4a727b8547b8fd2a0ff0b7718f1d65543c6e2777adf72
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [82]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from operator import itemgetter
import pandas as pd

In [83]:
def generate_recommendation(uid, model, dataset, thresh=4.5, amount=5):
    all_titles = list(dataset['iid'].values) # Все названия
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']  # ТЕ. что наш пользователь видел
    titles = np.array(list(set(all_titles) - set(users_seen_titles))) # ТО. что он не видел

    np.random.shuffle(titles)

    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title) # Прогноз полный
        rating = review_prediction.est # рейтинг из прогноза

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))
    return sorted(rec_list, key=itemgetter(1))[-5:]

    # return sorted(rec_list)[-5:]
            # if len(rec_list) >= amount:
            #     return rec_list

In [33]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [34]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [61]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [62]:
movies_with_ratings.shape

(100836, 6)

In [63]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [64]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [66]:
ratings.rating.min()

0.5

In [67]:
ratings.rating.max()

5.0

In [68]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [69]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [70]:
dataset['uid'].nunique(), dataset['iid'].nunique() # Определяем размеры таблиицы - к-во пользователй xк-во фильмов

(610, 9719)

#### user_based

In [71]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [73]:
generate_recommendation(2, algo, dataset, thresh=4.8)

[('Trinity and Sartana Are Coming (1972)', 5.0),
 ('Eddie Murphy Delirious (1983)', 5.0),
 ('Riki-Oh: The Story of Ricky (Lik Wong) (1991)', 5.0),
 ('Dead Meat (2004)', 5.0),
 ('The Adventures of Sherlock Holmes and Doctor Watson', 4.85)]

In [44]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8985


0.898482528669153

In [45]:
test_pred[:3] # Это просто 3 первые строки тестовой выборки с фактическим и предсказанным рейтингом

[Prediction(uid=542, iid='Kill Bill: Vol. 1 (2003)', r_ui=2.5, est=3.7592260572900007, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=469, iid='Bridge on the River Kwai, The (1957)', r_ui=4.0, est=4.168241951992395, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=317, iid='Scarface (1983)', r_ui=5.0, est=4.062599058241541, details={'actual_k': 49, 'was_impossible': False})]

In [46]:
algo.predict(uid=2, iid='Fight Club (1999)') # А это прогнозный рейтинг для фильма,iid='Fight Club (1999), который пользователь 2 еще не оценивал

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.474085079084162, details={'actual_k': 50, 'was_impossible': False})

#### item_based

In [47]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [48]:
test_pred = algo.test(testset)

In [49]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8982


0.898181158065249

In [50]:
new_pred = algo.predict(uid=2, iid='Fight Club (1999)')
new_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.2619917274785735, details={'actual_k': 24, 'was_impossible': False})

In [55]:
new_pred = algo.predict(uid=20000, iid='Fight Club (1999)')
new_pred

Prediction(uid=20000, iid='Fight Club (1999)', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [56]:
new_pred = algo.predict(uid=2, iid='unkown')
new_pred

Prediction(uid=2, iid='unkown', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [57]:
trainset.global_mean

3.498958694897605

#### Рекомендации

In [59]:
generate_recommendation(2, algo, dataset, thresh=4.8) # Здесь рекомендуются фильмы с рейтингом выше порога, необязательно он макисмальный

[('All the Vermeers in New York (1990)', 5.0),
 ('Mr. Skeffington (1944)', 4.91),
 ('When Worlds Collide (1951)', 5.0),
 ('Branded to Kill (Koroshi no rakuin) (1967)', 5.0),
 ('The Adventures of Sherlock Holmes and Doctor Watson: The Treasures of Agra (1983)',
  5.0)]

In [84]:
generate_recommendation(2, algo, dataset, thresh=4.8) # Здесь рекомендуются фильмы, которым пользователь поставил бы высокий рейтинг ( я функцию попправила)

[('Trinity and Sartana Are Coming (1972)', 5.0),
 ('Bill Burr: Why Do I Do This? (2008)', 5.0),
 ('Eva (2011)', 5.0),
 ('Dream of Light (a.k.a. Quince Tree Sun, The) (Sol del membrillo, El) (1992)',
  5.0),
 ('Vagabond (Sans toit ni loi) (1985)', 5.0)]